In [ ]:
! python --version
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from compMetabolomics.dash_egonet import run_network_visualization
from sklearn.metrics.pairwise import cosine_similarity
from compMetabolomics.construct_edge_list import construct_edge_list_for_cytoscape

In [ ]:

# Generate random data
data = np.random.rand(100, 100)

# Compute pairwise cosine similarity
similarity_matrix = cosine_similarity(data)

similarity_matrix.shape

# Static Representation using networkx defaults

In [ ]:
# Create a graph from the similarity matrix
graph = nx.from_numpy_array(similarity_matrix)

# Filter edges based on weight (similarity)
threshold = 0.8  # adjust this value to your needs
filtered_edges = [(node1, node2) for node1, node2, weight in graph.edges(data=True) if weight['weight'] < threshold]
graph.remove_edges_from(filtered_edges)

# remove self loops
self_loops = [(node, node) for node in graph.nodes()]
graph.remove_edges_from(self_loops)

# Draw the graph
nx.draw(graph, with_labels=True)
plt.show()

# Plotly Graph Objects Approach from networkX

In [ ]:

# Get positions of nodes using spring layout
pos = nx.spring_layout(graph)

# Create edge trace
edge_trace = go.Scatter(x=[], y=[], line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')

for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

# Create node trace
node_trace = go.Scatter(x=[], y=[], text=[], mode='markers', hoverinfo='text',
                        marker=dict(showscale=True, colorscale='YlGnBu', reversescale=True, color=[], size=10))

for node in graph.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

# Create figure
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(showlegend=False, hovermode='closest',
                              margin=dict(b=20, l=5, r=5, t=40),
                              xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                              yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

# Enable drag and zoom
fig.update_layout(dragmode='select', clickmode='event+select')

fig.show()

# Custom Cytoscape Dash network visualization - egonetwork

Click node to highlight node and its edges. 
Shift/ctrl/cmd click nodes for multi-selection and multi-highlight.
Shift-drag for rectangular select tool.
Click empty area or edge to remove highlighting.
Ids for selected features are printed to below the network.

In [ ]:
np.random.randint(0, 100, 1)[0]

In [ ]:

# Define the network data
# Generate random data
data = np.random.rand(100, 100)
top_k_max = 5
# Compute pairwise cosine similarity
similarity_matrix = cosine_similarity(data)

In [ ]:

nodes = [{
    'data': {
      'id': str(node_id), 
      'label': str(node_id), 
      'size' : np.random.randint(0, 100, 1)[0],
    },
    'position' : {'x': np.random.rand() * 1000, 'y': np.random.rand() * 1000}
  } for node_id in range(0, 100)]

print(nodes[0:3])


top_k_edge_list = construct_edge_list_for_cytoscape(similarity_matrix, [iloc for iloc in range(0, similarity_matrix.shape[0])], top_k= top_k_max)

# Making the edges more different in weight:
for edge in top_k_edge_list:
  edge["data"]["weight"] = np.random.random()
  edge["data"]["label"] = str(round(edge["data"]["weight"],2 ))
sorted_indices = np.array([elem["data"]["weight"] for elem in top_k_edge_list]).argsort()
top_k_edge_list = np.array(top_k_edge_list)[sorted_indices[::-1]].tolist()
print(top_k_edge_list[0:3])
# Initalize interactive app
app = run_network_visualization(top_k_edge_list + nodes)